In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import quartz as qz
from scipy.optimize import minimize
from quartz.api import *


from CAL.PyCAL import *
titleFont = font.copy()
titleFont.set_size(20)
legendFont = font.copy()
legendFont.set_size(14)


#*********************************************
# 因子模块
#*********************************************
_dataFrame = pd.read_excel('factorDescription.xlsx')
_dataFrame['type'] = map(lambda x: x[x.rfind(u'属于') :x.rfind(u'因子')][2:] + u'因子', _dataFrame[u'描述'])
_dataFrame.sort(columns = 'type', inplace= True)


factorDataFrame = _dataFrame[~(_dataFrame.type == u'因子')]
factorDataFrame['cnName'] = map(
    lambda x: x[:min(x.find(u'（') if x.find(u'（')>0 else 100, x.find(u'，') if x.find(u'，')>0 else 100,
                    x.find(u'。') if x.find(u'。')>0 else 100)],
    factorDataFrame[u'描述'])
factorDataFrame.index = range(len(factorDataFrame))


factorNameDict = factorDataFrame[[u'名称', 'cnName']].set_index(u'名称').cnName.to_dict()


classifier = {}
for tp in _dataFrame.type.drop_duplicates().values:
    if not tp == u'因子':
        classifier[tp] = _dataFrame[_dataFrame.type == tp]
        
        
def factorQuantileFilter(factor, secID, tradeDate, beginQuantile = 0.0, endQuantile = 0.2, 
                         dropNegtive = False, winsor = True, neutral = True, standard = True):
    df = DataAPI.MktStockFactorsOneDayGet(tradeDate, secID, field = 'ticker,secID,'+factor).set_index('ticker').sort(factor)
    if dropNegtive:
        df.drop(df[df[factor] < 0].index, inplace = True)
    df.dropna(axis = 'index', inplace = True)
    
    if df.empty:
        return df
    
    if isinstance(tradeDate, dt.datetime):
        tradeDate = tradeDate.strftime('%Y%m%d')
    
    if winsor:
        df[factor] = pd.Series(winsorize(df[factor].to_dict(), 'QuantileDraw'))
    if neutral:
        df[factor] = pd.Series(neutralize(df[factor].to_dict(), tradeDate))
    if standard:
        df[factor] = pd.Series(standardize(df[factor].to_dict()))
            
        
    return df[(df[factor] < df[factor].quantile(endQuantile)) & (df[factor] > df[factor].quantile(beginQuantile))].set_index('secID')


#*********************************************
# 股票信息
#*********************************************
def isST(stk, tradeDate):
    '''返回某只股票在tradeDate当天是不是ST'''
    secName = DataAPI.MktEqudGet(tradeDate = tradeDate, secID = stk, field = 'secShortName')['secShortName']
    if len(secName) > 0:
        return secName.str.contains('ST')[0]
    else:
        return -1


#*********************************************
# 行业相关
#*********************************************
industryList = [IndSW.CaiJueL1, IndSW.ChuanMeiL1, IndSW.DianQiSheBeiL1, IndSW.DianZiL1, IndSW.FangDiChanL1, IndSW.FangZhiFuZhuangL1,
IndSW.FeiYinJinRongL1, IndSW.GangTieL1, IndSW.GongYongShiYeL1, IndSW.GuoFangJunGongL1, IndSW.HuaGongL1, IndSW.JiSuanJiL1,
IndSW.JiXieSheBeiL1, IndSW.JiaYongDianQiL1, IndSW.JianZhuCaiLiaoL1, IndSW.JianZhuJianCaiL1, IndSW.JianZhuZhuangShiL1,
IndSW.JiaoTongYunShuL1, IndSW.JiaoYunSheBeiL1, IndSW.JinRongFuWuL1, IndSW.NongLinMuYuL1, IndSW.QiCheL1, IndSW.QingGongZhiZaoL1, 
IndSW.ShangYeMaoYiL1, IndSW.ShiPinYinLiaoL1, IndSW.TongXinL1, IndSW.XinXiFuWuL1, IndSW.XinXiSheBeiL1, IndSW.XiuXianFuWuL1,
IndSW.YiYaoShengWuL1, IndSW.YinHangL1, IndSW.YouSeJinShuL1, IndSW.ZongHeL1]

industryNameDict = DataAPI.EquIndustryGet(industry = u'申万')[['industryID1', 'industryName1']].drop_duplicates().set_index('industryID1').industryName1.to_dict()

industryDict = {industryNameDict[x.ind_code].decode('utf-8'): x for x in industryList}


#*********************************************
# 时间处理
#*********************************************
def timeSplit(beginDate, endDate, period):
    cal = Calendar('CHINA.SSE')
    timeSet = []
    tmpStart = beginDate
    tmpEnd = cal.advanceDate(tmpStart, '+'+str(period)+'B').strftime('%Y%m%d')
    while endDate > tmpEnd:
        timeSet += [(tmpStart, tmpEnd)]
        tmpStart = cal.advanceDate(tmpEnd, '+'+'1'+'B').strftime('%Y%m%d')
        tmpEnd = cal.advanceDate(tmpStart, '+'+str(period)+'B').strftime('%Y%m%d')
    timeSet += [(tmpStart, endDate)]
    return timeSet


#************************************************
# 快速回测表现报告
#************************************************
def reportBt(bt, perf):
    '''根据bt和perf画出回测累计收益及换手率情况'''
    if not bt.index.name == 'tradeDate':
        bt.set_index('tradeDate', inplace = True)
    bt['trade_vol'] = bt.blotter.apply(lambda x: sum([y.filled_amount*y.transact_price for y in x]))
    bt['buy_vol'] = bt.blotter.apply(lambda x: sum([y.filled_amount*y.transact_price if y.direction==1 else 0 for y in x]))
    bt['sell_vol'] = bt.blotter.apply(lambda x: sum([y.filled_amount*y.transact_price if y.direction==-1 else 0 for y in x]))

    data = pd.DataFrame()
    data['cumulative_returns'] = perf['cumulative_returns']
    data['benchmark_cumulative_returns'] = perf['benchmark_cumulative_returns']
    data['ex_cum_returns'] = data['cumulative_returns'] - data['benchmark_cumulative_returns']
    # data['stockPosition'] = 1 - bt.cash / bt.portfolio_value
    data.index = map(lambda x: x.strftime('%Y-%m-%d'), data.index)
    
    fig = plt.figure(figsize = (16, 10))    
    ax1 = fig.add_subplot(211)
    plt.title(u'回测累计收益及换手率情况', fontproperties = titleFont)
    data.plot(ax = ax1)
    # data['stockPosition'].plot(ax = ax1, secondary_y = True, label =  u'股票仓位')
    # plt.legend(prop = legendFont, loc = 2)
    # ax1.right_ax.set_ylim(0,1.1)
    ax1.legend([u'累计收益率',u'基准累计收益率',u'对冲后累计收益率'], prop = legendFont, loc = 0)
    ax1.grid(axis = 'y')
    # plt.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off') 
    
    ax2 = fig.add_subplot(212)
    tmpDf = (bt[['trade_vol', 'buy_vol', 'sell_vol']].T /bt.portfolio_value).T
    tmpDf['number'] = range(len(tmpDf))
    tmpDf = tmpDf[(tmpDf.trade_vol > 0) | (tmpDf.count == 0) | (tmpDf.count == len(tmpDf))]
    tmpDf.drop('number', axis = 1, inplace = True)
    tmpDf.index = map(lambda x: x.strftime('%Y-%m-%d'), tmpDf.index)
    barWidth = 0.1
    tmpDf.plot(kind = 'bar', ax = ax2)
    # fig.autofmt_xdate(rotation = 30, ha = 'right')
    plt.xticks(rotation = 30, ha = 'right')
    ax2.grid(axis = 'y')
    ax2.legend([u'交易换手率',u'买单换手率',u'卖单换手率'], prop = legendFont, loc = 0)
    plt.show()
    
    
def riskCoeffs(perf):
    '''根据perf给出回测的各个风险指标'''
    ex_value = perf['cumulative_returns'] - perf['benchmark_cumulative_returns'] + 1
    ex_returns = perf['returns'] - perf['benchmark_returns']
    
    perf['hedged_annualized_return'] = perf['annualized_return'] - perf['benchmark_annualized_return']
    perf['hedged_max_drawdown'] = max([1 - v/max(1, max(ex_value[:i+1])) for i,v in enumerate(ex_value)])
    perf['hedged_volatility'] = np.std(ex_returns) * np.sqrt(252)
    
    resultsDf = pd.Series(perf)
    resultsDf = resultsDf[[u'alpha', u'beta', u'information_ratio', u'sharpe', 
                            u'annualized_return', u'max_drawdown', u'volatility', 
                             u'hedged_annualized_return', u'hedged_max_drawdown', u'hedged_volatility']]
    resultsDf = pd.DataFrame(resultsDf).T
    resultsDf.index = ['风险系数']
    for col in resultsDf.columns:
        resultsDf[col] = [np.round(x, 3) for x in resultsDf[col]]

    cols = [(u'风险指标', u'Alpha'), (u'风险指标', u'Beta'), (u'风险指标', u'信息比率'), (u'风险指标', u'夏普比率'),
            (u'纯股票多头时', u'年化收益'), (u'纯股票多头时', u'最大回撤'), (u'纯股票多头时', u'收益波动率'), 
            (u'对冲后', u'年化收益'), (u'对冲后', u'最大回撤'), 
            (u'对冲后', u'收益波动率')]
    resultsDf.columns = pd.MultiIndex.from_tuples(cols)
    return resultsDf


def reportQuansBt(quansPerf):
    '''根据多个快速回测perf结果的dict画出累计收益及相对累计收益图像'''
    fig = plt.figure(figsize=(16,8))
    fig.set_tight_layout(True)
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)

    for qt in sorted(quansPerf.keys()):
        data = pd.DataFrame()
        data['cumulative_returns'] = quansPerf[qt]['cumulative_returns']
        data['benchmark_cumulative_returns'] = quansPerf[qt]['benchmark_cumulative_returns']
        data['excess_cumulative_returns'] = data.cumulative_returns - data.benchmark_cumulative_returns
        ax1.plot(data.index, data[['cumulative_returns']], label=str(qt))
        ax2.plot(data.index, data[['excess_cumulative_returns']], label=str(qt))


    ax1.legend(loc = 2)
    ax2.legend(loc = 2)
    ax1.set_ylabel(u"累计收益率", fontproperties=font, fontsize=16)
    ax2.set_ylabel(u"累计超额收益率", fontproperties=font, fontsize=16)
    ax1.set_title(u"不同分位数分组选股累计收益走势", fontproperties=font, fontsize=16)
    ax2.set_title(u"不同分位数分组选股对冲基准指数后累计收益走势", fontproperties=font, fontsize=16)
    ax1.grid(axis = 'y') ; ax2.grid(axis = 'y')
    plt.show()
    
    
def quansRiskCoeffs(quansPerf):
    '''根据多个快速回测perf结果的dict给出风险指标'''
    for qt in quansPerf:
        quansPerf[qt]['hedged_annualized_return'] = quansPerf[qt]['annualized_return'] - quansPerf[qt]['benchmark_annualized_return']
        ex_value = quansPerf[qt]['cumulative_returns'] - quansPerf[qt]['benchmark_cumulative_returns'] + 1
        tmp_ex_returns = quansPerf[qt]['returns'] - quansPerf[qt]['benchmark_returns']
        quansPerf[qt]['hedged_max_drawdown'] = max([1 - v/max(1, max(ex_value[:i+1])) for i,v in enumerate(ex_value)])
        quansPerf[qt]['hedged_volatility'] = np.std(tmp_ex_returns) * np.sqrt(252)

    resultsDf = pd.DataFrame(quansPerf).T.sort_index()

    resultsDf = resultsDf[[u'alpha', u'beta', u'information_ratio', u'sharpe', 
                            u'annualized_return', u'max_drawdown', u'volatility', 
                             u'hedged_annualized_return', u'hedged_max_drawdown', u'hedged_volatility']]

    for col in resultsDf.columns:
        resultsDf[col] = [np.round(x, 3) for x in resultsDf[col]]

    cols = [(u'风险指标', u'Alpha'), (u'风险指标', u'Beta'), (u'风险指标', u'信息比率'), (u'风险指标', u'夏普比率'),
            (u'纯股票多头时', u'年化收益'), (u'纯股票多头时', u'最大回撤'), (u'纯股票多头时', u'收益波动率'), 
            (u'对冲后', u'年化收益'), (u'对冲后', u'最大回撤'), 
            (u'对冲后', u'收益波动率')]
    resultsDf.columns = pd.MultiIndex.from_tuples(cols)
    resultsDf.index.name = u'分位组别'
    return resultsDf

#************************************************
# 组合优化有关
#************************************************

def get_covmat(tickers, date, periods):
    '''
    输入tickers + 日期 + 过去天数，获得以此计算出来的年化协方差矩阵
    '''
    start_date = shift_date(date, periods)
    return_mat=DataAPI.MktEqudAdjGet(ticker=tickers, beginDate=start_date, endDate=date, field=u"ticker,tradeDate,closePrice", pandas="1")
    return_mat = return_mat.pivot(index='tradeDate',columns='ticker', values='closePrice').pct_change().fillna(0.0)
    return return_mat.cov()*250

    
def get_smart_weight(cov_mat, method='min variance', wts_adjusted=False):
    '''
    功能：输入协方差矩阵，得到不同优化方法下的权重配置
    输入：
        cov_mat  pd.DataFrame,协方差矩阵，index和column均为资产名称
        method  优化方法，可选的有min variance、risk parity、max diversification、equal weight
    输出：
        pd.Series  index为资产名，values为weight
    PS:
        依赖scipy package
    '''
    
    if not isinstance(cov_mat, pd.DataFrame):
        raise ValueError('cov_mat should be pandas DataFrame！')
        
    omega = np.matrix(cov_mat.values)  # 协方差矩阵
    
    # 定义目标函数
    def fun1(x):
        return np.matrix(x) * omega * np.matrix(x).T
    
    def fun2(x):
        tmp = (omega * np.matrix(x).T).A1
        risk = x * tmp
        delta_risk = [sum((i - risk)**2) for i in risk]
        return sum(delta_risk)
    
    def fun3(x):
        den = x * omega.diagonal().T
        num = np.sqrt(np.matrix(x) * omega * np.matrix(x).T)
        return num/den
    
    # 初始值 + 约束条件 
    x0 = np.ones(omega.shape[0]) / omega.shape[0]  
    bnds = tuple((0,None) for x in x0)
    cons = ({'type':'eq', 'fun': lambda x: sum(x) - 1})
    options={'disp':False, 'maxiter':1000, 'ftol':1e-20}
        
    if method == 'min variance':   
        res = minimize(fun1, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options) 
    elif method == 'risk parity':
        res = minimize(fun2, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
    elif method == 'max diversification':
        res = minimize(fun3, x0, bounds=bnds, constraints=cons, method='SLSQP', options=options)
    elif method == 'equal weight':
        return pd.Series(index=cov_mat.index, data=1.0 / cov_mat.shape[0])
    else:
        raise ValueError('method should be min variance/risk parity/max diversification/equal weight！！！')
        
    # 权重调整
    if res['success'] == False:
        # print res['message']
        pass
    wts = pd.Series(index=cov_mat.index, data=res['x'])
    if wts_adjusted == True:
        wts = wts[wts >= 0.0001]
        return wts / wts.sum() * 1.0
    elif wts_adjusted == False:
        return wts
    else:
        raise ValueError('wts_adjusted should be True/False！')
        
        
def get_idx_cons(idx, date):
    '''
    功能：获取指数在某一天的成分股列表
    输入：
        idx 指数，xxxxxx型string，000300沪深300，000016上证50，000905中证500，000906中证800，000001上证综指
              可以为多个指数组合，写法为['xxxxxx','xxxxxx']，此时返回的结果已经去除重复的ticker了！！！
        date yyyymmdd型string
    输出：
        list of tickers
    依赖：
        DataAPI：IdxConsGet
    '''
    
    try:
        data = DataAPI.IdxConsGet(ticker=idx,intoDate=date,field='',pandas="1")['consTickerSymbol'].apply(lambda x: '%06d' % x)
    except:
        raise ValueError('DataAPI.IdxConsGet出错了！！！')

    return list(set(data))


def get_Atickers(date):
    '''
    功能：获取某一天的全部上市A股ticker（包括上市和暂停上市）
    输入：
        date，yyyymmdd日期
    输出：
        list of tickers
    依赖：
        DataAPI.EquGet
    '''
    
    try:
        universe = DataAPI.EquGet(equTypeCD=u"A",listStatusCD="L,S,DE",field=u"ticker,listDate,delistDate",pandas="1")
    except:
        raise ValueError('DataAPI.EquGet取数据异常！！！')
    universe['listdate'] = universe['listDate'].apply(lambda x: x.replace('-',''))  # 只取date之前上市的股票
    universe['delistdate'] = universe['delistDate'].apply(lambda x: x.replace('-','') if isinstance(x, str) else '99999999')
    universe = universe[(universe['listdate'] <= date) & (universe['delistdate'] > date)]
    tickers = universe['ticker'].tolist()
    return tickers
    
    
def get_halt_tickers(date, universe='A'):
    '''
    功能：获取某一天的全部/部分停牌股票（包括只在这一天某段时间停牌）
    输入：
        date，yyyymmdd型日期
        universe, 获取停牌股票的股票池，list，元素为tickers，默认为全A
    输出：
        list，这一天停牌的股票tickers
    依赖：
        需要用到DataAPI：SecHaltGet
        需要用到function：get_Atickers()
    '''
    
    if universe == 'A':
        universe = get_Atickers(date)
        return DataAPI.SecHaltGet(beginDate=date,endDate=date,ticker=universe,field=u"",pandas="1")['ticker'].tolist()
    elif isinstance(universe, list):
        return DataAPI.SecHaltGet(beginDate=date,endDate=date,ticker=universe,field=u"",pandas="1")['ticker'].tolist()
    else:
        raise ValueError('universe should be list with string tickers or string A！！！')


def get_dates(start_date, end_date, frequency='daily'):
    '''
    功能：输入起始日期和频率，即可获得日期列表（daily包括起始日，其余的都是位于起始日中间的）
    输入参数：
       start_date，开始日期，'xxxxxxxx'形式
       end_date，截止日期，'xxxxxxxx'形式
       frequency，频率，daily为所有交易日，daily1为所有自然日，weekly为每周最后一个交易日，weekly2为每隔两周，monthly为每月最后一个交易日，quarterly为每季最后一个交易日
    输出参数：
       获得list型日期列表，以'xxxxxxxx'形式存储
    PS:
        要用到DataAPI.TradeCalGet！！！
    '''
    
    data = DataAPI.TradeCalGet(exchangeCD=u"XSHG",beginDate=start_date,endDate=end_date,field=u"calendarDate,isOpen,isWeekEnd,isMonthEnd,isQuarterEnd",pandas="1")
    if frequency == 'daily':
        data = data[data['isOpen'] == 1]
    elif frequency == 'daily1':
        pass
    elif frequency == 'weekly':
        data = data[data['isWeekEnd'] == 1]
    elif frequency == 'weekly2':
        data = data[data['isWeekEnd'] == 1]
        data = data[0:data.shape[0]:2]
    elif frequency == 'monthly':
        data = data[data['isMonthEnd'] == 1]
    elif frequency == 'quarterly':
        data = data[data['isQuarterEnd'] == 1]
    else:
        raise ValueError('调仓频率必须为daily/daily1/weekly/weekly2/monthly/quarterly！！！')
    date_list = map(lambda x: x[0:4]+x[5:7]+x[8:10], data['calendarDate'].values.tolist())
    return date_list


def shift_date(date, n, direction='back'): 
    '''
    功能：给定date，获取该日期前/后n个交易日对应的交易日
    输入：
        date  'yyyymmdd'类型字符串
        n  非负整数，取值区间（0,720）
        direction  方向，取值为back/forward
    PS：
        get_dates()
    '''
    
    last_two_year = str(int(date[:4])-3) + '0101'
    forward_two_year = str(int(date[:4])+3) + '1231'
    if direction == 'back':
        date_list = get_dates(last_two_year, date, 'daily')
        return date_list[len(date_list)-1-n]
    elif direction == 'forward':
        date_list = get_dates(date, forward_two_year, 'daily')
        return date_list[n]
    else:
        raise ValueError('direction should be back/forward！！！')


def ticker2sec(ticker):
    '''
    功能：将ticker转换为secID，输入的ticker不要有重复值！
    输入：
        tickers：list、dict、series，list时没有value，dict/series时有对应的value
    输出：
        同输入类型
    依赖：
        需要用到DataAPI.EquGet
    细节：
        若没找到这个sec，则返回结果不包含这个ticker
    '''
    
    universe = DataAPI.EquGet(equTypeCD=u"A",listStatusCD="L,S,DE,UN",field=u"ticker,secID",pandas="1") # 获取所有的A股（包括已退市）
    universe = dict(universe.set_index('ticker')['secID'])
    if isinstance(ticker, list):
        res = []
        for i in ticker:
            if i in universe:
                res.append(universe[i])
            else:
                print i, ' 在universe中不存在，没有找到对应的secID！'
        return res
    elif isinstance(ticker, dict):
        res = {}
        for i in ticker:
            if i in universe:
                res[universe[i]] = ticker[i]
            else:
                print i, ' 在universe中不存在，没有找到对应的secID！'
        return res
    elif isinstance(ticker, pd.Series):
        res = {}
        for i in ticker.index:
            if i in universe:
                res[universe[i]] = ticker[i]
            else:
                print i, ' 在universe中不存在，没有找到对应的secID！'
        return pd.Series(res)
    else:
        raise ValueError('ticker should be list or dict or series！')
        
        
def sec2ticker(sec):
    '''
    功能：将sec转换为ticker，输入的sec不要有重复值！
    输入：
        sec：list、dict、series，list时没有value，dict/series时有对应的value
    输出：
        同输入类型
    依赖：
        需要用到DataAPI：EquGet
    细节：
        若没找到对应的ticker，则返回结果不包含这个sec
    '''
    
    universe = DataAPI.EquGet(equTypeCD=u"A",listStatusCD="L,S,DE,UN",field=u"ticker,secID",pandas="1") # 获取所有的A股（包括已退市）
    universe = universe[['secID','ticker']]
    universe = dict(universe.set_index('secID')['ticker'])
    if isinstance(sec, list):
        res = []
        for i in sec:
            if i in universe:
                res.append(universe[i])
            else:
                print i, ' 在universe中不存在，没有找到对应的ticker！'
        return res
    elif isinstance(sec, dict):
        res = {}
        for i in sec:
            if i in universe:
                res[universe[i]] = sec[i]
            else:
                print i, ' 在universe中不存在，没有找到对应的ticker！'
        return res
    elif isinstance(sec, pd.Series):
        res = {}
        for i in sec.index:
            if i in universe:
                res[universe[i]] = sec[i]
            else:
                print i, ' 在universe中不存在，没有找到对应的ticker！'
        return pd.Series(res)
    else:
        raise ValueError('sec should be list or dict or series！')
        
        
def cal_maxdrawdown(data):
    '''
    功能：给定净值数据（list, np.array, pd.Series, pd.DataFrame），返回最大回撤
    输入：
        data, list/np.array/pd.Series/pd.DataFrame，净值曲线，初始金为1
    输出：
        list/np.array/pd.Series返回float
        pd.DataFrame返回pd.DataFrame，index为DataFrame.columns
    '''
    
    if isinstance(data, list):
        data = np.array(data)
    if isinstance(data, pd.Series):
        data = data.values
        
    def get_mdd(values): # values为np.array的净值曲线，初始资金为1
        dd = [values[i:].min() / values[i] - 1 for i in range(len(values))]
        return abs(min(dd))
    
    if not isinstance(data, pd.DataFrame):
        return get_mdd(data)
    else:
        return data.apply(get_mdd)
    
    
def cal_indicators(df_daily_return):
    '''
    功能：给定daily return，计算各组合的评价指标，包括：年化收益率、年化标准差、夏普值、最大回撤
    输入：
        df_daily_return  pd.DataFrame，index为升序排列的日期，columns为各组合名称，value为daily_return
    '''
    
    df_cum_value = (df_daily_return + 1).cumprod()
    res = pd.DataFrame(index=['年化收益率','年化标准差','夏普值','最大回撤'], columns=df_daily_return.columns, data=0.0)
    res.loc['年化收益率'] = (df_daily_return.mean() * 250).apply(lambda x: '%.2f%%' % (x*100))
    res.loc['年化标准差'] = (df_daily_return.std() * np.sqrt(250)).apply(lambda x: '%.2f%%' % (x*100))
    res.loc['夏普值'] = (df_daily_return.mean() / df_daily_return.std() * np.sqrt(250)).apply(lambda x: np.round(x, 2))
    res.loc['最大回撤'] = cal_maxdrawdown(df_cum_value).apply(lambda x: '%.2f%%' % (x*100))
    return res


def cal_turnover(current_wts, target_wts):
    '''
   功能：给定当前持仓比例和目标持仓比例，计算双边换手率（默认持仓比例之和均为1）
   输入：
       current_wts，pd.Series，当前持仓比例
       target_wts，pd.Series，目标持仓比例
   输出：
       float
    '''
    if not isinstance(current_wts, pd.Series) or not isinstance(target_wts, pd.Series):
        raise ValueError('current_wts and target_wts should be pd.Series！！！')
    tmp = pd.merge(pd.DataFrame({'current_wts':current_wts}), pd.DataFrame({'target_wts':target_wts}), how='outer', left_index=True, right_index=True)
    tmp = tmp.fillna(0.0)
    tmp['turnover'] = tmp.current_wts - tmp.target_wts
    return abs(tmp['turnover']).sum()


def get_ticker_period_rtn(tickers, start_date, end_date):
    '''
    功能：输入tickers + 起始日期，获取tickers在这期间的daily return
    输入：
        ticker，list of ticker string
        start_date，yyyymmdd日期
        end_date，yyyymmdd日期
    输出：
        pd.DataFrame，index为日期yyyymmdd，columns为tickers string
    依赖：
        DataAPI：MktEqudAdjGet
        function：shift_date()
    '''
    begin_date = shift_date(start_date, 1)  # 向前推一天保证第一天也有daily return
    data = DataAPI.MktEqudAdjGet(ticker=tickers, beginDate=begin_date, endDate=end_date, field='ticker,tradeDate,closePrice', pandas='1')
    data['dates'] = data['tradeDate'].apply(lambda x: x.replace('-', ''))
    daily_rtn = data.pivot(index='dates', columns='ticker', values='closePrice').pct_change().fillna(0.0)
    return daily_rtn[daily_rtn.index >= start_date]

def get_oneday_cov(date, method='short'):
    '''
    获取某一天所有可取股票的方差协方差矩阵，值为年化后的真实值
    :param date: yyyymmdd
    :param method: 风险模型类型, day/short/long
    :return: pd.DataFrame, index和columns均为ticker
    '''

    expo = DataAPI.RMExposureDayGet(tradeDate=date)
    if len(expo) == 0:
        raise Exception('date should be trading date!!!')
    expo = expo.drop(['tradeDate', 'secID', 'exchangeCD', 'secShortName', 'updateTime'], axis=1).set_index('ticker')

    if method == 'day':
        fcov = DataAPI.RMCovarianceDayGet(tradeDate=date)
        fcov = fcov.drop(['tradeDate', 'FactorID', 'updateTime'], axis=1).set_index('Factor')
        srisk = DataAPI.RMSriskDayGet(tradeDate=date)
        srisk = srisk.drop(['tradeDate', 'secID', 'exchangeCD', 'secShortName', 'updateTime'],
                           axis=1).set_index('ticker')['SRISK']
    elif method == 'short':
        fcov = DataAPI.RMCovarianceShortGet(tradeDate=date)
        fcov = fcov.drop(['tradeDate', 'FactorID', 'updateTime'], axis=1).set_index('Factor')
        srisk = DataAPI.RMSriskShortGet(tradeDate=date)
        srisk = srisk.drop(['tradeDate', 'secID', 'exchangeCD', 'secShortName', 'updateTime'],
                           axis=1).set_index('ticker')['SRISK']
    else:
        fcov = DataAPI.RMCovarianceLongGet(tradeDate=date)
        fcov = fcov.drop(['tradeDate', 'FactorID', 'updateTime'], axis=1).set_index('Factor')
        srisk = DataAPI.RMSriskLongGet(tradeDate=date)
        srisk = srisk.drop(['tradeDate', 'secID', 'exchangeCD', 'secShortName', 'updateTime'],
                           axis=1).set_index('ticker')['SRISK']

    common_ticker = list(set(expo.index) & set(srisk.index))
    expo = expo.loc[common_ticker]
    srisk = srisk[common_ticker]

    fcov = fcov / 10000
    srisk = (srisk / 100) ** 2

    omega = np.matrix(expo.dot(fcov).dot(expo.T).values) + np.matrix(np.diag(srisk.values))
    res = pd.DataFrame(index=expo.index, columns=expo.index, data=omega)
    return res

def get_Auniverse(date):
    '''
    给定日期，获取这一天所有上市A股
    :param date: yyyymmdd string date
    :return: list of tickers
    '''

    universe = DataAPI.EquGet(equTypeCD=u"A", listStatusCD="L,S,DE", field=u"ticker,listDate,delistDate", pandas="1")
    universe['listdate'] = universe['listDate'].apply(lambda x: x.replace('-', ''))
    universe['delistdate'] = universe['delistDate'].apply(
        lambda x: x.replace('-', '') if isinstance(x, unicode) else '99999999')
    universe = universe[(universe['listdate'] <= date) & (universe['delistdate'] > date)]
    tickers = universe['ticker'].tolist()
    return tickers


def get_yugao(date):
    '''
    获取截止该日凌晨12点之前的所有业绩预告数据
    '''
    tickers = get_Auniverse(date)
    yugao = DataAPI.FdmtEfGet(beginDate='20170630', publishDateBegin=date, 
                              field='ticker,secShortName,actPubtime,NIncAPChgrLL,NIncAPChgrUPL,expnIncAPLL,expnIncAPUPL')
    yugao = yugao[yugao['ticker'].isin(tickers)]

    yugao = yugao.sort('actPubtime', ascending=False).set_index('ticker').drop_duplicates('secShortName')
    yugao['17Q1净利同比%'] = 0.0

    for i in yugao.index:

        # 17Q1净利润同比
        try:
            tmp = DataAPI.FdmtIndiGrowthPitGet(ticker=i, endDate='20170331', beginDate='20170331', field='niAttrPYOY')['niAttrPYOY'][0]
        except:
            tmp = np.nan
        yugao.loc[i, '17Q1净利同比%'] = tmp

        # 整合预告数据
        if np.isnan(yugao.loc[i, 'NIncAPChgrLL']):
            pre_niap = DataAPI.FdmtISGet(ticker=i, endDate='20160630', beginDate='20160630', field='NIncomeAttrP')['NIncomeAttrP'][0]
            yugao.loc[i, 'NIncAPChgrLL'] = 100.0 * yugao.loc[i, 'expnIncAPLL'] / pre_niap - 1
            yugao.loc[i, 'NIncAPChgrUPL'] = 100.0 * yugao.loc[i, 'expnIncAPUPL'] / pre_niap - 1
        else:
            continue

    # 一致预期
    yugao = yugao.rename(columns={'NIncAPChgrLL':'净利同比下限%', 'NIncAPChgrUPL':'净利同比上限%'}).drop(['expnIncAPLL','expnIncAPUPL'], axis=1)
    forecast = DataAPI.GG.CESTStockGGGet(BeginPubDate=date, EndPubDate=date, field='ticker,EPSType,rptDate,C7,C81')
    forecast = forecast[forecast['EPSType'] != 0].sort(['ticker', 'rptDate'], ascending=[True, True]).drop_duplicates(subset='ticker')
    forecast = forecast[forecast['ticker'].isin(tickers)]
    forecast['C7'] = forecast['C7'] * 100
    forecast = forecast.rename(columns={'C7':'一致预期净利同比%', 'C81':'4周一致预期变化%'}).drop(['EPSType','rptDate'], axis=1).set_index('ticker')
    yugao = pd.merge(yugao, forecast, how='left', left_index=True, right_index=True)
    return yugao

## 银行间质押式回购利率
def getHistDayInterestRateInterbankRepo(date):
    cal = Calendar('China.SSE')
    period = Period('-10B')
    begin = cal.advanceDate(date, period)
    begin_str = begin.toISO().replace('-', '')
    date_str = date.toISO().replace('-', '')
    # 以下的indicID分别对应的银行间质押式回购利率周期为：
    # 1D, 7D, 14D, 21D, 1M, 3M, 4M, 6M, 9M, 1Y
    indicID = [u"M120000067", u"M120000068", u"M120000069", u"M120000070", u"M120000071", 
               u"M120000072", u"M120000073", u"M120000074", u"M120000075", u"M120000076"]
    period = np.asarray([1.0, 7.0, 14.0, 21.0, 30.0, 90.0, 120.0, 180.0, 270.0, 360.0]) / 360.0
    period_matrix = pd.DataFrame(index=indicID, data=period, columns=['period'])
    field = u"indicID,indicName,publishTime,periodDate,dataValue,unit"
    interbank_repo = DataAPI.ChinaDataInterestRateInterbankRepoGet(indicID=indicID,beginDate=begin_str,endDate=date_str,field=field,pandas="1")
    interbank_repo = interbank_repo.groupby('indicID').first()
    interbank_repo = concat([interbank_repo, period_matrix], axis=1, join='inner').sort_index()
    return interbank_repo

## 银行间同业拆借利率
def getHistDaySHIBOR(date):
    cal = Calendar('China.SSE')
    period = Period('-10B')
    begin = cal.advanceDate(date, period)
    begin_str = begin.toISO().replace('-', '')
    date_str = date.toISO().replace('-', '')
    # 以下的indicID分别对应的SHIBOR周期为：
    # 1D, 7D, 14D, 1M, 3M, 6M, 9M, 1Y
    indicID = [u"M120000057", u"M120000058", u"M120000059", u"M120000060", 
               u"M120000061", u"M120000062", u"M120000063", u"M120000064"]
    period = np.asarray([1.0, 7.0, 14.0, 30.0, 90.0, 180.0, 270.0, 360.0]) / 360.0
    period_matrix = pd.DataFrame(index=indicID, data=period, columns=['period'])
    field = u"indicID,indicName,publishTime,periodDate,dataValue,unit"
    interest_shibor = DataAPI.ChinaDataInterestRateSHIBORGet(indicID=indicID,beginDate=begin_str,endDate=date_str,field=field,pandas="1")
    interest_shibor = interest_shibor.groupby('indicID').first()
    interest_shibor = concat([interest_shibor, period_matrix], axis=1, join='inner').sort_index()
    return interest_shibor

## 插值得到给定的周期的无风险利率
def periodsSplineRiskFreeInterestRate(date, periods):
    # 此处使用SHIBOR来插值
    init_shibor = getHistDaySHIBOR(date)
    
    shibor = {}
    min_period = min(init_shibor.period.values)
    min_period = 10.0/360.0
    max_period = max(init_shibor.period.values)
    for p in periods.keys():
        tmp = periods[p]
        if periods[p] > max_period:
            tmp = max_period * 0.99999
        elif periods[p] < min_period:
            tmp = min_period * 1.00001
        sh = interpolate.spline(init_shibor.period.values, init_shibor.dataValue.values, [tmp], order=3)
        shibor[p] = sh[0]/100.0
    return shibor

In [ ]:
"""
期权相关库
"""

from CAL.PyCAL import *
import numpy as np
import math
import seaborn as sns
from matplotlib import pylab
font.set_size(15)
import scipy as sp
from scipy.linalg import solve_banded

class BSMModel:
    def __init__(self, s0, r, sigma):
        self.s0 = s0
        self.x0 = math.log(s0)
        self.r = r
        self.sigma = sigma

    def log_expectation(self, T):
        return self.x0 + (self.r - 0.5 * self.sigma * self.sigma) * T

    def expectation(self, T):
        return self.s0 * math.exp(self.r * T)

    def x_max(self, T):
        return self.log_expectation(T) + 4.0 * self.sigma * math.sqrt(T)

    def x_min(self, T):
        return self.log_expectation(T) - 4.0 * self.sigma * math.sqrt(T) 
    
class CallOption:
    def __init__(self, strike):
        self.k = strike

    def ic(self, spot):
        return max(spot - self.k, 0.0)

    def bcl(self, spot, tau, model):
        return 0.0

    def bcr(self, spot, tau, model):
        return spot - math.exp(-model.r*tau) * self.k 

class BSMThetaScheme:
    def __init__(self, model, payoff, T, M, N, theta, xmin = None, xmax = None):
        self.model = model
        self.T = T
        self.M = M
        self.N = N
        self.dt = self.T / self.M
        
        self.payoff = payoff
        if xmin is None:
            self.x_min = model.x_min(self.T)
        else:
            self.x_min = xmin
            
        if xmax is None:
            self.x_max = model.x_max(self.T)
        else:
            self.x_max = xmax
        self.dx = (self.x_max - self.x_min) / self.N
        self.C = np.zeros((self.N+1, self.M+1)) # 全部网格
        self.xArray = np.linspace(self.x_min, self.x_max, self.N+1)
        self.C[:,0] = map(self.payoff.ic, np.exp(self.xArray))

        sigma_square = self.model.sigma*self.model.sigma
        r = self.model.r
        self.theta = theta

        # Implicit 部分
        self.l_j_implicit = -(1.0 - self.theta) * (0.5*sigma_square*self.dt/self.dx/self.dx - 0.5 * (r - 0.5 * sigma_square)*self.dt/self.dx)
        self.c_j_implicit = 1.0 + (1.0 - self.theta) * (sigma_square*self.dt/self.dx/self.dx + r*self.dt)
        self.u_j_implicit = -(1.0 - self.theta) * (0.5*sigma_square*self.dt/self.dx/self.dx + 0.5 * (r - 0.5 * sigma_square)*self.dt/self.dx)

        # Explicit 部分
        self.l_j_explicit = self.theta * (0.5*sigma_square*self.dt/self.dx/self.dx - 0.5 * (r - 0.5 * sigma_square)*self.dt/self.dx)
        self.c_j_explicit = 1.0 - theta * (sigma_square*self.dt/self.dx/self.dx + r*self.dt)
        self.u_j_explicit = self.theta * (0.5*sigma_square*self.dt/self.dx/self.dx + 0.5 * (r - 0.5 * sigma_square)*self.dt/self.dx)

    def roll_back(self): 

        for k in range(0, self.M):
            rhs = self._apply_explicit(k)
            self._apply_implicti(k,rhs)

    def _apply_explicit(self, k):
        # 应用显示格式部分
        preSol =  np.copy(self.C[:,k])
        rhs = np.array([0.0] * (self.N-1))

        for i in range(self.N-1):
            rhs[i] = self.l_j_explicit * preSol[i] + self.c_j_explicit * preSol[i+1] + self.u_j_explicit * preSol[i+2]
        return rhs

    def _apply_implicti(self, k, rhs):
        # 应用隐式格式部分
        udiag = np.ones(self.N-1) * self.u_j_implicit
        ldiag =  np.ones(self.N-1) * self.l_j_implicit
        cdiag =  np.ones(self.N-1) * self.c_j_implicit

        mat = np.zeros((3,self.N-1))
        mat[0,:] = udiag
        mat[1,:] = cdiag
        mat[2,:] = ldiag

        # 应用左端边值条件
        v1 = self.payoff.bcl(math.exp(self.x_min), (k+1)*self.dt, self.model)
        rhs[0] -= self.l_j_implicit * v1

        # 应用右端边值条件
        v2 = self.payoff.bcr(math.exp(self.x_max), (k+1)*self.dt, self.model)
        rhs[-1] -= self.u_j_implicit * v2

        x = solve_banded((1,1), mat, rhs)
        self.C[1:self.N, k+1] = x
        self.C[0][k+1] = v1
        self.C[self.N][k+1] = v2

    def mesh_grids(self):
        tArray = np.linspace(0, self.T, self.M+1)
        tGrids, xGrids = np.meshgrid(tArray, self.xArray)
        return tGrids, xGrids 
    
from CAL.PyCAL import *
from pandas import Series, DataFrame, concat
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style('white')
from matplotlib import pylab
import time
import math

def getHistDayOptions(var, date):
    # 使用DataAPI.OptGet，拿到已退市和上市的所有期权的基本信息；
    # 同时使用DataAPI.MktOptdGet，拿到历史上某一天的期权成交信息；
    # 返回历史上指定日期交易的所有期权信息，包括：
    # optID  varSecID  contractType  strikePrice  expDate  tradeDate  closePrice
    # 以optID为index。
    vixDateStr = date.toISO().replace('-', '')
    optionsMkt = DataAPI.MktOptdGet(tradeDate = vixDateStr, field = [u"optID", "tradeDate", "closePrice"], pandas = "1")
    optionsMkt = optionsMkt.set_index(u"optID")
    optionsMkt.closePrice.name = u"price"
    
    optionsID = map(str, optionsMkt.index.values.tolist())
    fieldNeeded = ["optID", u"varSecID", u'contractType', u'strikePrice', u'expDate']
    optionsInfo = DataAPI.OptGet(optID=optionsID, contractStatus = [u"DE", u"L"], field=fieldNeeded, pandas="1")
    optionsInfo = optionsInfo.set_index(u"optID")
    options = concat([optionsInfo, optionsMkt], axis=1, join='inner').sort_index()
    return options[options.varSecID==var]

def getNearNextOptExpDate(options, vixDate):
    # 找到options中的当月和次月期权到期日；
    # 用这两个期权隐含的未来波动率来插值计算未来30隐含波动率，是为市场恐慌指数VIX；
    # 如果options中的最近到期期权离到期日仅剩1天以内，则抛弃这一期权，改
    # 选择次月期权和次月期权之后第一个到期的期权来计算。
    # 返回的near和next就是用来计算VIX的两个期权的到期日
    optionsExpDate = Series(options.expDate.values.ravel()).unique().tolist()
    near = min(optionsExpDate)
    optionsExpDate.remove(near)
    if Date.parseISO(near) - vixDate < 1:
        near = min(optionsExpDate)
        optionsExpDate.remove(near)
    next = min(optionsExpDate)
    return near, next

def getStrikeMinCallMinusPutClosePrice(options):
    # options 中包括计算某日VIX的call和put两种期权，
    # 对每个行权价，计算相应的call和put的价格差的绝对值，
    # 返回这一价格差的绝对值最小的那个行权价，
    # 并返回该行权价对应的call和put期权价格的差
    call = options[options.contractType==u"CO"].set_index(u"strikePrice").sort_index()
    put  = options[options.contractType==u"PO"].set_index(u"strikePrice").sort_index()
    callMinusPut = call.closePrice - put.closePrice
    strike = abs(callMinusPut).idxmin()
    priceDiff = callMinusPut[strike]
    return strike, priceDiff

def calSigmaSquare(options, FF, R, T):
    # 计算某个到期日期权对于VIX的贡献sigma；
    # 输入为期权数据options，FF为forward index price，
    # R为无风险利率， T为期权剩余到期时间
    callAll = options[options.contractType==u"CO"].set_index(u"strikePrice").sort_index()
    putAll  = options[options.contractType==u"PO"].set_index(u"strikePrice").sort_index()
    callAll['deltaK'] = 0.05
    putAll['deltaK']  = 0.05
    
    # Interval between strike prices
    index = callAll.index
    if len(index) < 3:
        callAll['deltaK'] = index[-1] - index[0]
    else:
        for i in range(1,len(index)-1):
            callAll['deltaK'].ix[index[i]] = (index[i+1]-index[i-1])/2.0
        callAll['deltaK'].ix[index[0]] = index[1]-index[0]
        callAll['deltaK'].ix[index[-1]] = index[-1] - index[-2]
    index = putAll.index
    if len(index) < 3:
        putAll['deltaK'] = index[-1] - index[0]
    else:
        for i in range(1,len(index)-1):
            putAll['deltaK'].ix[index[i]] = (index[i+1]-index[i-1])/2.0
        putAll['deltaK'].ix[index[0]] = index[1]-index[0]
        putAll['deltaK'].ix[index[-1]] = index[-1] - index[-2]
    
    call = callAll[callAll.index > FF]
    put  = putAll[putAll.index < FF]
    FF_idx = FF
    if not put.empty:
        FF_idx = put.index[-1]
        put['closePrice'].iloc[-1] = (putAll.ix[FF_idx].closePrice + callAll.ix[FF_idx].closePrice)/2.0
        
    callComponent = call.closePrice*call.deltaK/call.index/call.index
    putComponent  = put.closePrice*put.deltaK/put.index/put.index
    sigma = (sum(callComponent)+sum(putComponent))*np.exp(T*R)*2/T
    sigma = sigma - (FF/FF_idx - 1)**2/T
    return sigma

def calDayVIX(optionVarSecID, vixDate):
    # 利用CBOE的计算方法，计算历史某一日的未来30日期权波动率指数VIX
   
    # The risk-free interest rates
    R_near = 0.06
    R_next = 0.06
    # 拿取所需期权信息
    options = getHistDayOptions(optionVarSecID, vixDate)
    termNearNext = getNearNextOptExpDate(options, vixDate)
    optionsNearTerm = options[options.expDate == termNearNext[0]]
    optionsNextTerm = options[options.expDate == termNearNext[1]]
    # time to expiration
    T_near = (Date.parseISO(termNearNext[0]) - vixDate)/365.0
    T_next = (Date.parseISO(termNearNext[1]) - vixDate)/365.0
    # the forward index prices
    nearPriceDiff = getStrikeMinCallMinusPutClosePrice(optionsNearTerm)
    nextPriceDiff = getStrikeMinCallMinusPutClosePrice(optionsNextTerm)
    near_F = nearPriceDiff[0] + np.exp(T_near*R_near)*nearPriceDiff[1]
    next_F = nextPriceDiff[0] + np.exp(T_next*R_next)*nextPriceDiff[1]
    # 计算不同到期日期权对于VIX的贡献
    near_sigma = calSigmaSquare(optionsNearTerm, near_F, R_near, T_near)
    next_sigma = calSigmaSquare(optionsNextTerm, next_F, R_next, T_next)

    # 利用两个不同到期日的期权对VIX的贡献sig1和sig2，
    # 已经相应的期权剩余到期时间T1和T2；
    # 差值得到并返回VIX指数(%)
    w = (T_next - 30.0/365.0)/(T_next - T_near)
    vix = T_near*w*near_sigma + T_next*(1 - w)*next_sigma
    return 100*np.sqrt(vix*365.0/30.0)

def getHistVIX(beginDate, endDate):
    # 计算历史一段时间内的VIX指数并返回
    optionVarSecID = u"510050.XSHG"
    cal = Calendar('China.SSE')
    dates = cal.bizDatesList(beginDate, endDate)
    dates = map(Date.toDateTime, dates)
    histVIX = pd.DataFrame(0.0, index=dates, columns=['VIX'])
    histVIX.index.name = 'date'
    for date in histVIX.index:
        histVIX['VIX'][date] =  calDayVIX(optionVarSecID, Date.fromDateTime(date))
    return histVIX

def getDayVIX(date):
    optionVarSecID = u"510050.XSHG"
    return calDayVIX(optionVarSecID, date)

## 计算一段时间标的的历史波动率，返回值包括以下不同周期的波动率：
# 一周，半月，一个月，两个月，三个月，四个月，五个月，半年，九个月，一年，两年
# https://uqer.datayes.com/v3/community/share/560493a4f9f06c597565ef03
def getHistVolatilityEWMA(secID, beginDate, endDate):
    cal = Calendar('China.SSE')
    spotBeginDate = cal.advanceDate(beginDate,'-520B',BizDayConvention.Preceding)
    spotBeginDate = Date(2006, 1, 1)
    begin = spotBeginDate.toISO().replace('-', '')
    end = endDate.toISO().replace('-', '')
    
    fields = ['tradeDate', 'preClosePrice', 'closePrice', 'settlePrice', 'preSettlePrice']
    security = DataAPI.MktFunddGet(secID=secID, beginDate=begin, endDate=end, field=fields)
    security['dailyReturn'] = security['closePrice']/security['preClosePrice']   # 日回报率
    security['u2'] = (np.log(security['dailyReturn']))**2      # u2为复利形式的日回报率平方
    # security['u2'] = (security['dailyReturn'] - 1.0)**2      # u2为日价格变化百分比的平方
    security['tradeDate'] = pd.to_datetime(security['tradeDate'])
    
    periods = {'hv1W': 5, 'hv2W': 10, 'hv1M': 21, 'hv2M': 41, 'hv3M': 62, 'hv4M': 83, 
               'hv5M': 104, 'hv6M': 124, 'hv9M': 186, 'hv1Y': 249, 'hv2Y': 497}
    # 利用pandas中的ewma模型计算波动率
    for prd in periods.keys():
        # 此处的span实际上就是上面计算波动率公式中lambda表达式中的N
        security[prd] = np.round(np.sqrt(pd.ewma(security['u2'], span=periods[prd], adjust=False)), 5)*math.sqrt(252.0)
    
    security = security[security.tradeDate >= beginDate.toISO()]
    security = security.set_index('tradeDate')
    return security

## 计算一段时间标的的历史波动率，返回值包括以下不同周期的波动率：
# 一周，半月，一个月，两个月，三个月，四个月，五个月，半年，九个月，一年，两年
def getHistVolatilityC2C(secID, beginDate, endDate):
    cal = Calendar('China.SSE')
    spotBeginDate = cal.advanceDate(beginDate,'-520B',BizDayConvention.Preceding)
    spotBeginDate = Date(2006, 1, 1)
    begin = spotBeginDate.toISO().replace('-', '')
    end = endDate.toISO().replace('-', '')
    
    fields = ['tradeDate', 'preClosePrice', 'closePrice', 'settlePrice', 'preSettlePrice']
    security = DataAPI.MktFunddGet(secID=secID, beginDate=begin, endDate=end, field=fields)
    security['dailyReturn'] = security['closePrice']/security['preClosePrice']    # 日回报率
    security['u'] = np.log(security['dailyReturn'])      # u2为复利形式的日回报率
    security['tradeDate'] = pd.to_datetime(security['tradeDate'])
    
    periods = {'hv1W': 5, 'hv2W': 10, 'hv1M': 21, 'hv2M': 41, 'hv3M': 62, 'hv4M': 83, 
               'hv5M': 104, 'hv6M': 124, 'hv9M': 186, 'hv1Y': 249, 'hv2Y': 497}
    # 利用方差模型计算波动率
    for prd in periods.keys():
        security[prd] = np.round(pd.rolling_std(security['u'], window=periods[prd]), 5)*math.sqrt(252.0)
    
    security = security[security.tradeDate >= beginDate.toISO()]
    security = security.set_index('tradeDate')
    return security

#  看跌看涨成交量（成交额）比率 PCR
#  https://uqer.datayes.com/v3/community/share/561c883df9f06c4ca72fb5f7
def histVolumeOpt50ETF(beginDate, endDate):
    ## 计算历史一段时间内的50ETF期权持仓量交易量数据
    
    optionVarSecID = u"510050.XSHG"
    cal = Calendar('China.SSE')
    dates = cal.bizDatesList(beginDate, endDate)
    dates = map(Date.toDateTime, dates)
    columns = ['callVol', 'putVol', 'callValue',   
               'putValue', 'callOpenInt', 'putOpenInt',
               'nearCallVol', 'nearPutVol', 'nearCallValue', 
               'nearPutValue', 'nearCallOpenInt', 'nearPutOpenInt',
               'netVol', 'netValue', 'netOpenInt',
               'volPCR', 'valuePCR', 'openIntPCR', 
               'nearVolPCR', 'nearValuePCR', 'nearOpenIntPCR']
    hist_opt = pd.DataFrame(0.0, index=dates, columns=columns)
    hist_opt.index.name = 'date'
    # 每一个交易日数据单独计算
    for date in hist_opt.index:
        date_str = Date.fromDateTime(date).toISO().replace('-', '')
        try:
            opt_data = DataAPI.MktOptdGet(secID=u"", tradeDate=date_str, field=u"", pandas="1")
        except:
            hist_opt = hist_opt.drop(date)
            continue
        
        opt_type = []
        exp_date = []
        for ticker in opt_data.secID.values:
            opt_type.append(ticker[6])
            exp_date.append(ticker[7:11])
        opt_data['optType'] = opt_type
        opt_data['expDate'] = exp_date
        near_exp = np.sort(opt_data.expDate.unique())[0]
        
        data = opt_data.groupby('optType')
        # 计算所有上市期权：看涨看跌交易量、看涨看跌交易额、看涨看跌持仓量
        hist_opt['callVol'][date] = data.turnoverVol.sum()['C']
        hist_opt['putVol'][date] = data.turnoverVol.sum()['P']
        hist_opt['callValue'][date] = data.turnoverValue.sum()['C']
        hist_opt['putValue'][date] = data.turnoverValue.sum()['P']
        hist_opt['callOpenInt'][date] = data.openInt.sum()['C']
        hist_opt['putOpenInt'][date] = data.openInt.sum()['P']
        
        near_data = opt_data[opt_data.expDate == near_exp]
        near_data = near_data.groupby('optType')
        # 计算近月期权(主力合约)： 看涨看跌交易量、看涨看跌交易额、看涨看跌持仓量
        hist_opt['nearCallVol'][date] = near_data.turnoverVol.sum()['C']
        hist_opt['nearPutVol'][date] = near_data.turnoverVol.sum()['P']
        hist_opt['nearCallValue'][date] = near_data.turnoverValue.sum()['C']
        hist_opt['nearPutValue'][date] = near_data.turnoverValue.sum()['P']
        hist_opt['nearCallOpenInt'][date] = near_data.openInt.sum()['C']
        hist_opt['nearPutOpenInt'][date] = near_data.openInt.sum()['P']
        
        # 计算所有上市期权： 总交易量、总交易额、总持仓量
        hist_opt['netVol'][date] = hist_opt['callVol'][date] + hist_opt['putVol'][date]
        hist_opt['netValue'][date] = hist_opt['callValue'][date] + hist_opt['putValue'][date]
        hist_opt['netOpenInt'][date] = hist_opt['callOpenInt'][date] + hist_opt['putOpenInt'][date]
        
        # 计算期权看跌看涨期权交易量(持仓量)的比率：
        # 交易量看跌看涨比率，交易额看跌看涨比率, 持仓量看跌看涨比率
        # 近月期权交易量看跌看涨比率，近月期权交易额看跌看涨比率, 近月期权持仓量看跌看涨比率
        # PCR = Put Call Ratio
        hist_opt['volPCR'][date] = round(hist_opt['putVol'][date]*1.0/hist_opt['callVol'][date], 4)
        hist_opt['valuePCR'][date] = round(hist_opt['putValue'][date]*1.0/hist_opt['callValue'][date], 4)
        hist_opt['openIntPCR'][date] = round(hist_opt['putOpenInt'][date]*1.0/hist_opt['callOpenInt'][date], 4)
        hist_opt['nearVolPCR'][date] = round(hist_opt['nearPutVol'][date]*1.0/hist_opt['nearCallVol'][date], 4)
        hist_opt['nearValuePCR'][date] = round(hist_opt['nearPutValue'][date]*1.0/hist_opt['nearCallValue'][date], 4)
        hist_opt['nearOpenIntPCR'][date] = round(hist_opt['nearPutOpenInt'][date]*1.0/hist_opt['nearCallOpenInt'][date], 4)
    return hist_opt

def histPrice50ETF(beginDate, endDate):
    # 华夏上证50ETF收盘价数据
    secID = '510050.XSHG'
    begin = Date.fromDateTime(beginDate).toISO().replace('-', '')
    end = Date.fromDateTime(endDate).toISO().replace('-', '')
    fields = ['tradeDate', 'closePrice', 'preClosePrice']
    etf = DataAPI.MktFunddGet(secID, beginDate=begin, endDate=end, field=fields)
    etf['tradeDate'] = pd.to_datetime(etf['tradeDate'])
    etf['dailyReturn'] = etf['closePrice'] / etf['preClosePrice'] - 1.0 
    etf = etf.set_index('tradeDate')
    return etf

def histPCR50ETF(beginDate, endDate):
    # PCRD: Put Call Ratio Diff
    # 计算每日PCR变化量：当日PCR减去前一日PCR得到的值，即对PCR做差分
    # 专注于某一项PCR，例如：成交额PCR --- valuePCR
    pcr_names = ['volPCR', 'valuePCR', 'openIntPCR', 
                 'nearVolPCR', 'nearValuePCR', 'nearOpenIntPCR']
    pcr_diff_names = [pcr + 'Diff' for pcr in pcr_names]
    pcr = histVolumeOpt50ETF(beginDate, endDate)
    for pcr_name in pcr_names:
        pcr[pcr_name + 'Diff'] = pcr[pcr_name].diff()
    return pcr[pcr_names + pcr_diff_names]
    